**Our GPU**

In [1]:
!nvidia-smi

Tue May 27 19:26:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [44]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
%%writefile omp.cpp

#define MAX 10
#include<stdio.h>
#include<omp.h>

int main(){
  int count = 0;
  #pragma omp parallel num_threads(MAX)
  {
    #pragma omp atomic update
    count++;
  }
  printf("Number of threads: %d\n", count);
}

Overwriting omp.cpp


In [ ]:
!g++ -fopenmp omp.cpp -o omp
!./omp

Number of threads: 10


In [49]:
%%writefile ani.cpp

#include<stdio.h>
#include<omp.h>
#include<stdlib.h>
#define n 10

int main(){
  int maximum = 0;
  int z[n];

  for(int j=0; j<n; j++){
    z[j] = rand();
  }

  printf("The elements of Array are: \n");

  for(int j=0; j<n; j++){
    printf("%d\n", z[j]);
  }

  #pragma omp parallel for
  for(int j=0; j<n; j++){
    if(z[j]>maximum){
      maximum = z[j];
    }
  }

  return 0;

}

Overwriting ani.cpp


In [50]:
!g++ -fopenmp ani.cpp -o ani
!./ani

The elements of Array are: 
1804289383
846930886
1681692777
1714636915
1957747793
424238335
719885386
1649760492
596516649
1189641421


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-0lcx5wz7
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-0lcx5wz7
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10742 sha256=fdb30b442a03a2b978d94f242f4f3839c3b517111d4c5bcc31496f40b5aacbba
  Stored in directory: /tmp/pip-ephem-wheel-cache-k349xc7b/wheels/ef/1d/c6/f7e47f1aa1bc9d05c4120d94f90a79cf28603ef343b0dd43ff
Successfully built nvcc4jupyter


In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp1d46952t".


In [ ]:
%%cuda
#include <iostream>

__global__ void hello_cuda() {
    printf("Hello from GPU thread %d!\n", threadIdx.x);
}

int main() {
    std::cout << "Hello from CPU!\n";
    hello_cuda<<<1, 10>>>(); // Launch 1 block, 10 threads
    cudaDeviceSynchronize(); // Wait for GPU to finish
    return 0;
}

Hello from CPU!



In [ ]:
 %%writefile three.cpp

#include<iostream>

void some_func(void)
{
    int i;
    int a[128];
    int b[128];
    int c[128];

    for (i=0;i<128;i++)
    {
      b[i] = i+1;
      c[i] = i * 2;
      a[i] = b[i] * c[i];
    }

    std::cout << "Contents of array a:" << std::endl;
    for (i=0; i<128; i++) {
        std::cout << a[i] << " ";
    }
    std::cout << std::endl;
}

int main() {
  some_func();
  return 0;
}

Overwriting three.cpp


In [ ]:
!g++ -fopenmp three.cpp -o three
!./three

Contents of array a:
0 4 12 24 40 60 84 112 144 180 220 264 312 364 420 480 544 612 684 760 840 924 1012 1104 1200 1300 1404 1512 1624 1740 1860 1984 2112 2244 2380 2520 2664 2812 2964 3120 3280 3444 3612 3784 3960 4140 4324 4512 4704 4900 5100 5304 5512 5724 5940 6160 6384 6612 6844 7080 7320 7564 7812 8064 8320 8580 8844 9112 9384 9660 9940 10224 10512 10804 11100 11400 11704 12012 12324 12640 12960 13284 13612 13944 14280 14620 14964 15312 15664 16020 16380 16744 17112 17484 17860 18240 18624 19012 19404 19800 20200 20604 21012 21424 21840 22260 22684 23112 23544 23980 24420 24864 25312 25764 26220 26680 27144 27612 28084 28560 29040 29524 30012 30504 31000 31500 32004 32512 


In [ ]:
%%writefile four.cpp

#include <iostream>
#include <omp.h>

__global__
void some_kernel_func (int * const a, int * const b, int * const c, int n)
{
  a[i] = b[i] * c[i];
}

int main() {
  int n = 128;
  int a[n];
  int b[n];
  int c[n];
}

Overwriting four.cpp


In [ ]:
!gcc -fopenmp four.cpp -o four
!./four

cc1plus: fatal error: four.cpp: No such file or directory
compilation terminated.
/bin/bash: line 1: ./four: No such file or directory


In [12]:
%%writefile five.cpp

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

// #include "common.h"

#define SHARED_ARRAY_SIZE 128

__global__ void smem_static_test(int * in, int * out, int size)
{
	int tid = threadIdx.x;
	int gid = blockIdx.x * blockDim.x + threadIdx.x;

	__shared__ int smem[SHARED_ARRAY_SIZE];

	if (gid < size)
	{
		smem[tid] = in[gid];
		out[gid] = smem[tid];
	}
}

__global__ void smem_dynamic_test(int * in, int * out, int size)
{
	int tid = threadIdx.x;
	int gid = blockIdx.x * blockDim.x + threadIdx.x;

	extern __shared__ int smem[];

	if (gid < size)
	{
		smem[tid] = in[gid];
		out[gid] = smem[tid];
	}
}


Overwriting five.cpp


In [13]:
!nvcc five.cpp -o five

/usr/bin/ld: /tmp/tmpxft_0000411a_00000000-5_five.o: warning: relocation against `blockDim' in read-only section `.text'
/usr/bin/ld: /usr/lib/gcc/x86_64-linux-gnu/11/../../../x86_64-linux-gnu/Scrt1.o: in function `_start':
(.text+0x1b): undefined reference to `main'
/usr/bin/ld: /tmp/tmpxft_0000411a_00000000-5_five.o: in function `smem_static_test(int*, int*, int)':
five.cpp:(.text+0x34): undefined reference to `threadIdx'
/usr/bin/ld: five.cpp:(.text+0x40): undefined reference to `blockIdx'
/usr/bin/ld: five.cpp:(.text+0x46): undefined reference to `blockDim'
/usr/bin/ld: five.cpp:(.text+0x4f): undefined reference to `threadIdx'
/usr/bin/ld: /tmp/tmpxft_0000411a_00000000-5_five.o: in function `smem_dynamic_test(int*, int*, int)':
five.cpp:(.text+0xeb): undefined reference to `threadIdx'
/usr/bin/ld: five.cpp:(.text+0xf4): undefined reference to `blockIdx'
/usr/bin/ld: five.cpp:(.text+0xfa): undefined reference to `blockDim'
/usr/bin/ld: five.cpp:(.text+0x103): undefined reference to 

In [14]:
!./five

/bin/bash: line 1: ./five: No such file or directory


**Usecase of Multiple Pointers**

In [15]:
%%writefile six.cpp

#include<stdio.h>

int main(){
  int value = 42;
  int *ptr1 = &value;
  int **ptr2 = &ptr1;
  int ***ptr3 = &ptr2;

  printf("Value of value: %d\n", ***ptr3);
}

Writing six.cpp


In [16]:
!gcc six.cpp -o six

In [17]:
!./six

Value of value: 42


**Use of Void Pointers**

In [18]:
%%writefile seven.cpp

#include <stdio.h>

int main() {
    int num = 10;
    float fnum = 13.21;
    void* ptr;

    ptr = &num;
    printf("Value of num: %d\n", *((int *)ptr));

    ptr = &fnum;
    printf("Value of fnum: %f\n", *((float *)ptr));

    return 0;
}

Writing seven.cpp


In [19]:
!gcc seven.cpp -o seven

In [20]:
!./seven

Value of num: 10
Value of fnum: 13.210000


In [26]:
%%writefile eight.cpp

#include <stdio.h>
#include <stdlib.h>

int main(){
  int* ptr = NULL;
  printf("1. initial ptr value: %p\n ", (void*)ptr);

  if(ptr=NULL){
    printf("2. ptr is NULL\n");
  }

  ptr = (int*)malloc(sizeof(int));
  if(ptr == NULL){
    printf("3. memory allocation falied\n");
    return 1;
  }
}

Overwriting eight.cpp


In [27]:
!gcc eight.cpp -o eight

In [28]:
!./eight

1. initial ptr value: (nil)
 

In [39]:
%%writefile nine.cpp

#include <stdio.h>

int main(){
  int arr[] = {13,21,34,52};
  int* ptr = arr;

  printf("1. postiion one: %d\n", *ptr);
  for(int i=0; i<4; i++){
    printf("%d", *ptr);
    printf("%p\n", ptr);
    ptr++;
  }
}

Overwriting nine.cpp


In [40]:
!gcc nine.cpp -o nine

In [42]:
!./nine

1. postiion one: 13
130x7ffd29cfd440
210x7ffd29cfd444
340x7ffd29cfd448
520x7ffd29cfd44c


In [1]:
!nvidia --version

/bin/bash: line 1: nvidia: command not found
